<a href="https://colab.research.google.com/github/marichaf/msgs-colab-wfs/blob/main/MSGs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Measurable skills gain
# Created by Maricha, 11/18/24


In [ ]:
# This block is required for use on Google Drive only. DO NOT EDIT

from google.colab import drive

drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # not using this, potentially remove
from datetime import datetime
from tabulate import tabulate

## Edit the cell below where noted. ONLY edit the cell below!
After you edit the cell with the file names and date, in the menu above, under "Runtime", click Run all.

You may see a dialog box pop up asking your permission to mount this to your Google drive. Click ok and only give it the required permissions.

In [ ]:
### THIS IS THE ONLY CELL THAT REQUIRES EDITING TO RUN THE SCRPT! ###

# Read the CSV file into a pandas dataframe
shared_drive_path = '/content/drive/Shared drives/Service Delivery/ETO Reports/Python Scripts/MSGs/' # Only update if the filepath has changed

# INSERT FILE NAMES HERE: Keep apostrophes and .csv. Make sure it matches exactly.
itss_report = 'ITSS_10_1_20 to 9_30_23.csv'
msg_report = 'MSG_10_1_2022 to 9_30_2023.csv'
program_enrollments = 'PER_1_1_2020 to 9_30_2023.csv'

# UPDATE start and end date of the cohort period (per the PIRL reporting timeline)
# We need this to check for valid ITAs and Credential Attainments later.
cohort_start_date = '2022-10-01' # UPDATE: format needs to be yyyy-mm-dd, and keep the apostrophes
cohort_end_date = '2023-09-30' # UPDATE: format needs to be yyyy-mm-dd, and keep the apostrophes

# UPDATE report date and cohort quarter date - just for filenames.
# This will be pulled in to the filename that gets exported, so the format matters but won't break anything if you change it (it can be different if we want)
report_date = '3_27_25'  # UPDATE to the date you are running this/the date of the data you pull.
quarter = 'PY23 MSG Q3 Post' # Decide if we want to use dates or quarters on output file names




In [ ]:
file_path_itss = f'{shared_drive_path}{itss_report}'
data = pd.read_csv(file_path_itss, low_memory=False, encoding='utf-8', encoding_errors='replace')
print(f'ITSS report has {data.shape[0]} rows and {data.shape[1]} columns')

file_path_msg = f'{shared_drive_path}{msg_report}'
msg = pd.read_csv(file_path_msg, low_memory=False, encoding='utf-8', encoding_errors='replace')
print(f'MSG report has {msg.shape[0]} rows and {msg.shape[1]} columns')


file_path_enrollments = f'{shared_drive_path}{program_enrollments}'
enrollments = pd.read_csv(file_path_enrollments, low_memory=False, encoding='utf-8', encoding_errors='replace')
print(f'Program Enrollments report has {enrollments.shape[0]} rows and {enrollments.shape[1]} columns')

# Define the date range of cohort period
start_date = pd.Timestamp(cohort_start_date)  # Start date (inclusive)
end_date = pd.Timestamp(cohort_end_date) # end date (inclusive) (this used to be today's date but for consistency I just made it the end date)

ITSS report has 12789 rows and 38 columns
MSG report has 6960 rows and 79 columns
Program Enrollments report has 6960 rows and 79 columns


In [ ]:
mask = data.applymap(lambda x: isinstance(x, str) and '�' in x)
rows_with_replacement_char = data[mask.any(axis=1)]

<ipython-input-38-d33cf207d221>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = data.applymap(lambda x: isinstance(x, str) and '�' in x)


In [ ]:
rows_with_replacement_char

,Is Youth Program,Program Category,WDA Name,Office Name,Last Updated By,Created By,Date Entered,Date Created,Date Last Updated,Opted Out of Data Sharing?,...,Unique Enrollment ID,Program Of Enrollment,Program Enrollment Status,Program Enrollment Start Date,Program Enrollment End Date,Local Program on Program Enrollment TP,Contract (if applicable) on Program Enrollment TP,ETO Office Number,O*NET Code_14584,Staff Restricted


In [ ]:
problem_columns = mask.any().loc[lambda x: x].index.tolist()
print("Columns with replacement character:", problem_columns)

Columns with replacement character: []


In [ ]:
# Program enrollments - to see all coenrollments and filter correctly
# I only need to aggregate on Program Name and maybe keep Enrollment Start Date and Enrollment End Date

fields_to_aggregate_enrollments = ['Program Name',
                                   'Enrollment Start Date',
                                   'Enrollment End Date']

def aggregate_column(values):
    """
    Aggregates a column by:
    - Converting all values to strings.
    - Removing duplicates.
    - Sorting the values.
    - Joining them into a single string separated by "; ".
    """
    # Convert all values to strings
    values_as_strings = map(str, values)
    # Remove duplicates
    unique_values = set(values_as_strings)
    # Sort the values
    sorted_values = sorted(unique_values)
    # Join the sorted values into a single string
    result = "; ".join(sorted_values)
    return result


# Define the aggregation logic dictionary
agg_logic_enrollments = {field: aggregate_column for field in fields_to_aggregate_enrollments}

# Group by 'ETO Case Number' and apply the aggregation logic
enrollments_agg = enrollments.groupby('ETO Case Number').agg(agg_logic_enrollments).reset_index()
enrollments_agg.shape

(5635, 4)

In [ ]:
enrollments_agg

,ETO Case Number,Program Name,Enrollment Start Date,Enrollment End Date
0,210,WIOA Adult,12/30/2022,12/30/2022
1,448,WIOA Adult,7/21/2023,7/21/2023
2,532,WIOA Adult,4/5/2023,5/4/2023
3,918,WIOA Adult,9/21/2022,10/13/2022
4,991,WIOA Adult,5/5/2023,5/30/2023
...,...,...,...,...
5630,2599407,WIOA Adult,9/26/2023,10/16/2023
5631,2601374,State-funded Economic Security for All (EcSA),9/15/2023,12/27/2024
5632,2602415,State-funded Economic Security for All (EcSA),9/15/2023,12/11/2023
5633,2603681,WIOA Adult,9/26/2023; 9/28/2023,9/27/2024


In [ ]:
# Temporary, for debugging
#enrollments_agg.to_csv(f'{shared_drive_path}enrollments_agg_{report_date}.csv', index=False) # just testing, remove

In [ ]:
# Need to merge enrollments with ITSS data. CHECK THERE ARE NO LOSSES. Then only keep ones where one of the WIOA programs is present the list of program enrollments.
itss_programs = data.merge(
    enrollments_agg[["ETO Case Number", "Program Name", 'Enrollment Start Date', 'Enrollment End Date']],
    on="ETO Case Number",
    how="left"
)

# Check number of rows pre and post merge
print(f'{enrollments_agg.shape=}')
print(f'{data.shape=}')
print(f'{itss_programs.shape=}')

enrollments_agg.shape=(5635, 4)
data.shape=(12789, 38)
itss_programs.shape=(12789, 41)


In [ ]:
# Establish list of qualifying training services:
services = ['Adult Alternative High School Diploma/GED',
            'Alternative Secondary School Services or Dropout Recovery Services (Youth Only)',
            'Apprenticeship Training',
            'Customized Training (2.0)',
            'EDUCATIONAL ACHIEVEMENT SERVICES (YOUTH ONLY)',
            'Entrepreneurial Training',
            'Entrepreneurial Training (2.0)',
            'Increased Capacity Training',
            'Incumbent Worker',
            'Occupational Skills Training',
            'Occupational Skills Training (2.0)',
            'Occupational Skills Training (Youth Only)',
            'On-the-Job Training (2.0)',
            'TAA Approved Training',
            'TUTORING, STUDY SKILLS TRAINING, INSTRUCTION, AND DROPOUT PREVENTION (YOUTH ONLY)',
            'Workplace Training with Related Instruction Workplace Training with Related Instruction'
           ]

# and job seeker does not have an "other" reason for exit (Neutral exit, will also need to merge with outcomes report later for this)

# Establish list of programs to filter
programs = ['WIOA Adult',
            'WIOA Dislocated Worker',
            'WIOA Out of School Youth',
            'WIOA In School Youth']

In [ ]:
# Looking at all the column names for ITSS report
list(itss_programs.columns)

['Is Youth Program',
 'Program Category',
 'WDA Name',
 'Office Name',
 'Last Updated By',
 'Created By',
 'Date Entered',
 'Date Created',
 'Date Last Updated',
 'Opted Out of Data Sharing?',
 'Name',
 'ETO Case Number',
 'Unique Record ID',
 'Service Participation Episode ID',
 'Service Description',
 'Method of Contact',
 'Projected activity start date',
 'Projected activity end date',
 'Activity Start Date',
 'Activity End Date',
 'Actual Outcome',
 'Contract',
 'Contract (if not listed above)',
 'Training Provider',
 'Provider Name (if not listed above)',
 'Course',
 'Other Program',
 'Service is Associated with a Program Enrollment TP?',
 'Unique Enrollment ID',
 'Program Of Enrollment',
 'Program Enrollment Status',
 'Program Enrollment Start Date',
 'Program Enrollment End Date',
 'Local Program on Program Enrollment TP',
 'Contract (if applicable) on Program Enrollment TP',
 'ETO Office Number',
 'O*NET Code_14584',
 'Staff Restricted',
 'Program Name',
 'Enrollment Start Date

In [ ]:
# Filter ITS TP report; keep all those with one of the qualifying services, and is one of our programs.
# This filtering becomes the Denominator list (before removing "neutral" exits which we will do later).

#OLD version: #denominator = itss_programs.loc[itss_programs['Program Name'].isin(programs) & itss_programs['Service Description'].isin(services)]

# Create a regex pattern to match any of the program names
programs_pattern = '|'.join([f"\\b{program}\\b" for program in programs])

# Filter rows where 'Program Name' contains any of the specified programs
# THIS NEEDS EDITING. All ISY are automatically included in the denominator.
  # ALTHOUGH PIRL data is not currently doing this!

denominator = itss_programs[
    itss_programs['Program Name'].str.contains(programs_pattern, na=False) &
    itss_programs['Service Description'].isin(services)
]

'''

# Updated version (keeping old version for now, don't  need all ISYs until WAWorks comes online):
denominator = itss_programs[
    itss_programs['Program Name'].str.contains(programs_pattern, na=False) &
    (
        (itss_programs['Program Name'] == 'WIOA In School Youth') |
        (itss_programs['Service Description'].isin(services))
    )
]

'''
# Finally, we also need to remove any qualifying training services that ended before the cohort period.
  # If they are not actively in a training program in this period, they are not in the denominator.

  # Cleaning strings of dates, since apparently there is some issue when converting to datetime...
denominator['Activity End Date'] = denominator['Activity End Date'].astype(str).str.strip()
denominator['Activity End Date'] = pd.to_datetime(denominator['Activity End Date'], errors='coerce')

filtered_denominator = denominator[
    ((denominator['Activity End Date'].isnull()) |
    (denominator['Activity End Date'] >= end_date))
]

<ipython-input-47-eb67daf4d66b>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  denominator['Activity End Date'] = denominator['Activity End Date'].astype(str).str.strip()
<ipython-input-47-eb67daf4d66b>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  denominator['Activity End Date'] = pd.to_datetime(denominator['Activity End Date'], errors='coerce')


In [ ]:
# Temporary, for debugging
#denominator.to_csv(f'{shared_drive_path}denominator_{report_date}.csv', index=False) # just testing, remove

In [ ]:
filtered_denominator.shape

(114, 41)

In [ ]:
# Just checking, visualizing snippet of the dataset
filtered_denominator[['Service Description', 'Program Of Enrollment', 'Program Name']]


,Service Description,Program Of Enrollment,Program Name
6,Occupational Skills Training (2.0),QUEST NDWG,Economic Security for All (EcSA); QUEST NDWG; ...
1336,Occupational Skills Training (2.0),QUEST NDWG,QUEST NDWG; WIOA Adult
1492,Occupational Skills Training (2.0),QUEST NDWG,COVID-19 Disaster Recovery DWG; COVID-19 Emplo...
1709,Occupational Skills Training (2.0),COVID-19 Employment Recovery DWG,COVID-19 Disaster Recovery DWG; COVID-19 Emplo...
1867,Occupational Skills Training (2.0),COVID-19 Employment Recovery DWG,COVID-19 Employment Recovery DWG; WIOA Adult; ...
...,...,...,...
11821,Occupational Skills Training,NaN,WIOA Adult
12309,Alternative Secondary School Services or Dropo...,NaN,WIOA Out of School Youth
12311,Youth Guidance and Counseling,NaN,WIOA In School Youth
12315,Alternative Secondary School Services or Dropo...,NaN,WIOA In School Youth


## Categorizing Training Outcomes, to count credential attainments as MSGs

This section copies over some code from the Credential Attainment script to check for Credential Attainments that occured within the MSG reporting period, in case they were not captured as an MSG.

*Note, we may need to consider the possibility of double counting if they were reported as MSG AND Credential Attainment, but I think the later handling of duplicates would account for this (since we count people, not # of MSGs or CAs).

In [ ]:
# copy from CA, but using my denominator dataset

# Replace blanks with "no value"
filtered_denominator['Actual Outcome'] = filtered_denominator['Actual Outcome'].fillna('No Value').replace('', 'No Value')
# NOTE: Local Services report optimized uses the field "Service Outcome". ITS report uses fied "Actual Outcome".
# But both reports should get you the same answer.

## NOTE: Services list for CA is different from the one for MSGs...

## NOTE 2: Not sure we want to do anything about duplicates yet, but copied this over from the CA script...

# Dictionary mapping services to expected appropriate outcomes
outcome_mapping = {
    'Positive training outcome': ['Associates Degree Earned',
                     'Attained GED/HS Equivalency',
                     'Attained High School Diploma',
                     'Bachelors Degree Earned',
                     'Locally Authorized Credential Earned',
                     'Masters Or Doctorate Degree Earned',
                     'Occupational Skills Certificate Or Credential Earned',
                    ],
    'Incorrect training outcome': ['Not Required For This Service',
                                   'OJT Completed Not Hired',
                                   'OJT Hired By Non Training Related Employer',
                                   'OJT Hired by OJT Employer',
                                   'OJT Hired By Other Training Related Employer',
                                   'OJT Successful Completion',
                                   'Work Readiness Credential Earned',
                                   'Younger Youth Skill Goal Attained',
                                   'Younger Youth Skill Goal Not Attained',
                                   'No Value'
                                  ],
    'Expected training outcome': ['Associates Degree Earned',
                                  'Attained GED/HS Equivalency',
                                  'Attained High School Diploma',
                                  'Bachelors Degree Earned',
                                  'Locally Authorized Credential Earned',
                                  'Masters Or Doctorate Degree Earned',
                                  'Occupational Skills Certificate Or Credential Earned',
                                  'Completed-Certificate/Credential Pending',
                                  'Completed-No Credential/Certificate',
                                  'Did Not Complete',
                                  'Unknown Outcome - Historical Data Cleanup',
                                 ]
}

# Function to categorize based on service description and actual outcome
def categorize_service(row):
    service = row['Service Description']
    outcome = row['Actual Outcome'] # change Actual to Service and vv depending on the report used

    if service in services: # for each service in the qualifying training services list (see above)
        if outcome in outcome_mapping['Positive training outcome']: # and has a positive credential outcome
            """
            if service == "Alternative Secondary School Services or Dropout Recovery Services (Youth Only)":
                return 'check for employment/post-secondary' # trigger manual check of employment (for now, automate later)
                """ # we don't need this here since HSD/GED attainment counts for MSG independently of employment, however we may want to flag it.
            return 'Positive ITA outcome' # goes in the numerator
        if outcome in outcome_mapping['Incorrect training outcome']: # and has an incorrect outcome for a training service
            return 'Data entry error: training service requires training-related outcome' # trigger return to subs
        if outcome == "Completed-No Credential/Certificate":
            return 'Data check/possible error: credential or certificate expected for a completed training service'
        return 'Negative ITA outcome' # service is a qualifying service but has negative outcome (not in positive outcome list) and not incorrect outcome

    # The below section shouldn't really apply since we should have already filtered all the services prior to this point
    else: # if service not in qualifying service list (i.e., all other services)
        if outcome in outcome_mapping['Positive training outcome']: # and has a positive training outcome
            if service == "Training Paid By Other":
                return 'Does CA count for MSGs count when Training Paid by Other?'
            return 'Data entry error: positive training outcome for non-training service'
        if outcome == "Completed-Certificate/Credential Pending":
            if service == "Training Paid By Other":
                return 'Not considered in CA calculation'
            return 'Data entry error: pending credential for for non-training service'
        else: # service is not a qualifying training service and outcome is also not a positive training outcome
            return 'Not considered in CA calculation'


# Apply the function to create a new column categorizing the services
filtered_denominator['CA screen result'] = filtered_denominator.apply(categorize_service, axis=1)

# Now you can group by Actual Outcome and remove duplicate ETO Case Numbers as needed
def filter_duplicates(group):
    duplicated_cases = group[group.duplicated(subset=['ETO Case Number'], keep=False)]

    if not duplicated_cases.empty:
        keep_rows = duplicated_cases['Service Description'].isin(services)
        return group[keep_rows | ~group['ETO Case Number'].duplicated(keep='first')]

    return group


<ipython-input-51-f7841441591c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_denominator['Actual Outcome'] = filtered_denominator['Actual Outcome'].fillna('No Value').replace('', 'No Value')
<ipython-input-51-f7841441591c>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_denominator['CA screen result'] = filtered_denominator.apply(categorize_service, axis=1)


## Merging with MSG report


In [ ]:
# Bring in MSG report
# (Moved to top cell to bring both reports in at the same time, easier for shared script)

# MSG dataset is called 'msg'



In [ ]:
# Just looking at all the column names
list(msg.columns)

['Is Youth Program',
 'Program Category',
 'ETO Case Number',
 'Name Detail',
 'Is Opted-Out',
 'Enrollment Status',
 'Program Name',
 'Local Program Name',
 'WDA',
 'Office',
 'Enrollment Start Date',
 'Enrollment End Date',
 'Date Created',
 'TP Originally Created By',
 'TP Created on Behalf Of',
 'Household income greater than 200% of Federal Poverty Level',
 'Gender',
 'Age',
 'Date of Birth',
 'Disabled',
 'Low Income',
 'Date of Actual Dislocation',
 'Contract (if applicable) internal ETO value',
 'Contract Name',
 'In School',
 'In School Type',
 'Homeless',
 'Ethnicity',
 'Authorized Worker',
 'Authorized Worker_TAA',
 'Attended Rapid Response Event',
 'Rapid Response Event Number',
 'Employment Status',
 'Basic Literacy Skills Deficiency',
 'Basic Literacy Skills Deficiency_TAA',
 'Youth Offender - first one on the PE form',
 'Selective Service',
 'Dropout',
 'Dropout_TAA',
 'Runaway',
 'Do You Have Limited Proficiency in English?',
 'Currently Employed',
 'Did you become unem

In [ ]:
# Checking for repeated ETO Numbers across datasets (this is expected, just checking)
etos_denominator = filtered_denominator['ETO Case Number'].nunique()
msg_unique = msg['Case Number'].nunique()
print(f"{etos_denominator=} {filtered_denominator.shape[0]=}")
print(f"{msg_unique=}  {msg.shape[0]=}")

# There are repeat ETO numbers in both dfs, MSGs having more repeats.

KeyError: 'Case Number'

In [ ]:
print(msg['Date of progress report'].dropna().unique()[:20])

In [ ]:
# Cleaning strings of dates, since apparently there is some issue when converting to datetime...
msg['Date of progress report'] = msg['Date of progress report'].astype(str).str.strip()
msg['Date of progress report'] = pd.to_datetime(msg['Date of progress report'], errors='coerce')


In [ ]:
# MSG report dates (the dates you chose when pulling the report in ETO) refer to the date it was entered.
# We need to filter the report to only keep MSGs that actually happened during the time frame, using "Date of progress report".

msg['Date of progress report'] = pd.to_datetime(msg['Date of progress report'], format='%m/%d/%y') # formatting to datetime for filtering

In [ ]:


# Filter to keep only rows within the range
# PROBEM: There are blank "Date of progress report". BIG ISSUE FOR DATA ENTRY VALIDATION!"
filtered_msg = msg[
    (msg['Date of progress report'].isnull()) |
    ((msg['Date of progress report'] >= start_date) & (msg['Date of progress report'] <= end_date))
]
# Had to edit to include blank 'Date of of progress report'. THIS IS A PROBLEM. Need to know which PIRL potentially uses. This could account for a lot of the disconnect.
filtered_msg.shape

In [ ]:
filtered_msg.loc[:, 'ETO Case Number'] = filtered_msg['Case Number'] # renaming this field to match ITS TP report for merging

In [ ]:
# Merge date filtered MSG report with denominator dataset (filtered from ITS TP report)

filtered_denominator['source_df'] = 'ITS_filtered_denominator'
filtered_msg['source_df'] = 'msg_report'

merged_df_with_duplicates = filtered_msg.merge(
    filtered_denominator, # You can specify which columns you want to pass in, or all
    on='ETO Case Number',
    how='outer',  # Use 'outer' if you want to capture all rows from both DataFrames
    indicator=True  # Adds the _merge column to indicate merge status
)

# Replace default labels in the _merge column to something we can understand
merged_df_with_duplicates['_merge'] = merged_df_with_duplicates['_merge'].cat.rename_categories({
        'left_only': 'MSG only',
        'right_only': 'Denominator only',
        'both': 'Both'
})

In [ ]:
# Trying the merge a different way, aggregating duplicates first (we want to keep all rows since each may have meaninful information.

fields_to_aggregate = [
    'Unique Record ID',
    'Service Participation Episode ID',
    'Service Description',
    'Activity Start Date',
    'Activity End Date',
    'Actual Outcome',
    'Training Provider',
    'Course',
    'Other Program',
    'Program Of Enrollment',
    'Program Enrollment Start Date',
    'Program Name',
    'Enrollment Start Date',
    'Enrollment End Date',
    'CA screen result'
]

fields_to_aggregate_msg = ['Record Type',
 'ProgramofEnrollment',
 'Date Taken',
 'Date of progress report',
 'ProgressReportType',
 'ProgressReportDate',
 'Istheprogresssatisfactory',
 'Did the participant successfully pass an exam that is required for a particular occupation, or progress in attaining technical or occupational skills as evidenced by trade-related benchmarks such as knowledge-based exams?_14671',
 'Date of report card or transcript',
 'TranscriptDate',
 'DegreeType',
 'MeetsAcademicStandards',
 'Test Date',
 'PreorPostTest',
 'Assessment Category_158',
 'Category of Assessment Verification_4360',
 'Type of Assessment Test_157',
 'Please Specify Other Type of Assessment_160',
 'Functional Area_156',
 'Other Functional Area_3186',
 'Test Date_2840',
 'Grade_13305',
 'RawCorrect',
 'RawIncorrect',
 'Educational Functioning Level_3189',
 'Basic Skills Deficient_3190'
                          ]

def aggregate_column(values):
    """
    Aggregates a column by:
    - Converting all values to strings.
    - Removing duplicates.
    - Sorting the values.
    - Joining them into a single string separated by "; ".
    """
    # Convert all values to strings
    values_as_strings = map(str, values)
    # Remove duplicates
    unique_values = set(values_as_strings)
    # Sort the values
    sorted_values = sorted(unique_values)
    # Join the sorted values into a single string
    result = "; ".join(sorted_values)
    return result


# Define the aggregation logic dictionary
agg_logic_denominator = {field: aggregate_column for field in fields_to_aggregate}

# Group by 'ETO Case Number' and apply the aggregation logic
denominator_agg = filtered_denominator.groupby('ETO Case Number').agg(agg_logic_denominator).reset_index()


## Same for filtered_msg:
agg_logic_msg = {field: aggregate_column for field in fields_to_aggregate_msg}

msg_agg = filtered_msg.groupby('ETO Case Number').agg(agg_logic_msg).reset_index()

# Perform the outer merge
merged_df = pd.merge(denominator_agg, msg_agg, on='ETO Case Number', how='outer', indicator=True, suffixes=('_ITS', '_MSG'))
# Replace default labels in the _merge column to something we can understand
merged_df['_merge'] = merged_df['_merge'].cat.rename_categories({
        'left_only': 'Denominator: Training but no MSG',
        'right_only': 'Flag: MSG Test and Results but no Training TP',
        'both': 'Numerator: Training and MSG'
})


In [ ]:
# Need to add another step that screens the MSGs, there are some cases where an MSG is reported but progress is NOT satisfactory.
# These should not count in the numerator.
# NOTE: slighltly problematic that there are MSG TPs where "Istheprogresssatisfactory" is blank. We can only eliminate Nos, we can't assume anything about blanks.

# Update "_merge" where "Istheprogresssatisfactory" contains "No" - should be "contains" and not "is" since they could be aggregated
merged_df.loc[merged_df['Istheprogresssatisfactory'].str.contains('No', na=False), '_merge'] = 'Denominator: Training but no MSG'


In [ ]:
# Adding another step to capture a successfull Credential Attainment that occurs within the period, where there is no MSG.

# From previous work, we assume "Activity End Date" falls within the reporting period. (This is how ETO filters the MSGs report)

# Update "_merge" where "CA screen result" contains "Positive ITA outcome" - should be "contains" and not "is" since they could be aggregated
merged_df.loc[merged_df['CA screen result'].str.contains('Positive ITA outcome', na=False), '_merge'] = 'Numerator: Training and MSG'
  # Consider making a different category because they don't have an MSG, just credential attainment.


In [ ]:
 # Just visualizing
print(f'{denominator_agg.shape=}')
print(f'{msg_agg.shape=}')
print(f'{merged_df.shape=}')

# Just checking
merged_df


In [ ]:
# Count unique ETO Case Numbers for each _merge category
unique_counts = merged_df.groupby('_merge', observed=True)['ETO Case Number'].nunique()

print(unique_counts)

In [ ]:
# Same as above, but by program enrollment

# This is a lot less helpful now that I have the full list of program enrollments per person.
# nested_counts = merged_df.groupby(['Program Name', '_merge'], observed=True)['ETO Case Number'].nunique()

# print(nested_counts)


# Explode "Program Name" to create one row per program
merged_df_exploded = merged_df.assign(
    Program_Name_Split=merged_df['Program Name'].str.split(';')
).explode('Program_Name_Split')

# Strip whitespace from program names
merged_df_exploded['Program_Name_Split'] = merged_df_exploded['Program_Name_Split'].str.strip()

# Filter to keep only the programs in your subset list
filtered_programs = merged_df_exploded[
    merged_df_exploded['Program_Name_Split'].isin(programs)
]

# Group by the individual program and '_merge' to count unique ETO Case Numbers
nested_counts = filtered_programs.groupby(
    ['Program_Name_Split', '_merge'], observed=True
)['ETO Case Number'].nunique()

print(nested_counts)


In [ ]:
# merged_df_exploded.to_csv(f'{shared_drive_path}TESTMSG_report_{report_date}.csv', index=False)

# Reset index to turn the grouped data into a DataFrame
nested_counts_df = nested_counts.reset_index()

# Display the table using tabulate
#print(tabulate(nested_counts_df, headers='keys', tablefmt='grid'))

# nested_counts_df.to_csv(f'{shared_drive_path}MSG_nested_counts_{report_date}.csv', index=False))

In [ ]:

# Create a new DataFrame from the grouped data
nested_counts_df = nested_counts.reset_index()

# Pivot the data to separate Numerator and Denominator counts
pivoted_table = nested_counts_df.pivot(
    index='Program_Name_Split',
    columns='_merge',
    values='ETO Case Number'
).fillna(0)

# Rename columns for clarity (adjust based on your `_merge` values)
pivoted_table.rename(columns={
    'left_only': 'Denominator: Training but no MSG',
    'both': 'Numerator: Training and MSG',
}, inplace=True)

# Calculate the True Denominator and Percentage
pivoted_table['True Denominator'] = pivoted_table['Denominator: Training but no MSG'] + pivoted_table['Numerator: Training and MSG']
pivoted_table['Percentage'] = pivoted_table['Numerator: Training and MSG'] / pivoted_table['True Denominator'] * 100

# Reset the index for exporting or display
final_table = pivoted_table.reset_index()

# Display the table using tabulate
from tabulate import tabulate
print(tabulate(final_table, headers='keys', tablefmt='grid'))

# Export to CSV or Excel
# final_table.to_csv(f'{shared_drive_path}MSG_program_counts_separate_{report_date}.csv', index=False)



In [ ]:
# Combine the two youth rows into a single row
# Pivot the data to separate Numerator and Denominator counts
pivoted_table_combo = nested_counts_df.pivot(
    index='Program_Name_Split',
    columns='_merge',
    values='ETO Case Number'
).fillna(0)

combined_youth = pivoted_table_combo.loc[['WIOA In School Youth', 'WIOA Out of School Youth']].sum()

# Create a new row for "WIOA Youth"
pivoted_table_combo.loc['WIOA Youth'] = combined_youth

# Drop the original youth rows
pivoted_table_combo.drop(['WIOA In School Youth', 'WIOA Out of School Youth'], inplace=True)

# Recalculate the True Denominator and Percentage for the updated table
pivoted_table_combo['True Denominator'] = pivoted_table_combo['Denominator: Training but no MSG'] + pivoted_table_combo['Numerator: Training and MSG']
pivoted_table_combo['Percentage'] = pivoted_table_combo['Numerator: Training and MSG'] / pivoted_table_combo['True Denominator'] * 100

# Reset the index for exporting or display
final_combined_table = pivoted_table_combo.reset_index()

# Display the table using tabulate
print(tabulate(final_combined_table, headers='keys', tablefmt='grid'))

# Export to CSV or Excel
final_combined_table.to_csv(f'{shared_drive_path}MSG_program_counts_{report_date}_{quarter}.csv', index=False)

In [ ]:
# Just reordering columns for easier visualization in Excel
# This method seems unnecessarily complicated (thanks ChatGPT lol), consider simplifying when you have time :)

# Get the list of columns
columns = merged_df.columns.tolist()

# Find the position of "_merge" and move "CA screen result" before it
merge_index = columns.index('_merge')  # Find the position of _merge
columns.remove('CA screen result')     # Remove 'CA screen result' temporarily
columns.insert(merge_index - 1, 'CA screen result')  # Insert it right before _merge

# Reorder the DataFrame
merged_df = merged_df[columns]

# Check the result
# print(merged_df.head())

In [ ]:
merged_df.to_csv(f'{shared_drive_path}MSG_report_{report_date}_{quarter}.csv', index=False) # we can rename this file if we want